# Eventformer: Frame-Free Vision Transformer

Run on Google Colab with T4 GPU. Go to **Runtime > Change runtime type > T4 GPU**

In [ ]:
# 1. Check GPU
!nvidia-smi
import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

In [ ]:
# 2. Clone repository
!rm -rf /content/Eventformer
!git clone https://github.com/jkinarthur/Eventformer.git
%cd /content/Eventformer/code

In [ ]:
# 3. Install dependencies
!pip install -q einops timm scipy

In [ ]:
# 4. Setup imports
import sys
sys.path.insert(0, '/content/Eventformer/code')
sys.path.insert(0, '.')

import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# Import models
from models import eventformer_tiny, EventformerForClassification
from models.ctpe import ContinuousTimePositionalEncoding
from models.paaa import PolarityAwareAsymmetricAttention
from models.asna import ASNABlock

print('All imports successful!')

In [ ]:
# 5. Test Model
model = eventformer_tiny().to(device)
num_params = sum(p.numel() for p in model.parameters())
print('Parameters:', f'{num_params:,}')

# Test forward pass
B, N = 4, 2048
coords = torch.rand(B, N, 2, device=device) * 346
times = torch.rand(B, N, device=device).sort(dim=1)[0]
pols = torch.randint(0, 2, (B, N), device=device).float() * 2 - 1

with torch.no_grad():
    out = model(coords, times, pols)

print('Output shape:', out[0].shape)
print('Model test passed!')

In [ ]:
# 6. Generate Publication Figures
import os
os.makedirs('figures', exist_ok=True)

from visualize import generate_all_figures
figures = generate_all_figures(output_dir='figures')

In [ ]:
# 7. Display Figures
from IPython.display import Image, display
import os

figure_files = [
    'training_curves.png',
    'ablation_all_datasets.png',
    'ablation_detailed.png',
    'comparison_accuracy_vs_flops.png',
    'comparison_accuracy_vs_params.png',
    'velocity_stratification.png',
    'delta_t_sensitivity.png',
    'event_cloud_visualization.png',
    'fps_comparison.png'
]

for fname in figure_files:
    fpath = os.path.join('figures', fname)
    if os.path.exists(fpath):
        print(f'\n--- {fname} ---')
        display(Image(filename=fpath, width=700))

In [ ]:
# 8. Show LaTeX Tables
print('='*60)
print('LATEX TABLES')
print('='*60)

if os.path.exists('figures/all_tables.tex'):
    with open('figures/all_tables.tex', 'r') as f:
        print(f.read())

In [ ]:
# 9. Download all figures
!zip -r eventformer_results.zip figures/

from google.colab import files
files.download('eventformer_results.zip')
print('Download ready!')